In [1]:
import numpy as np
import time
from numba import njit

In [2]:
NUM_OF_STARTING_CARDS = 6

SUN_SCORES_ARRAY = np.zeros(14, dtype=np.uint8)
SUN_SCORES_ARRAY[1]  = 11
SUN_SCORES_ARRAY[7]  = 0
SUN_SCORES_ARRAY[8]  = 0
SUN_SCORES_ARRAY[9]  = 0
SUN_SCORES_ARRAY[10] = 10
SUN_SCORES_ARRAY[11] = 2
SUN_SCORES_ARRAY[12] = 3
SUN_SCORES_ARRAY[13] = 4

SUN_RANK_ORDER = np.zeros(14, dtype=np.uint8)
SUN_RANK_ORDER[1] = 8
SUN_RANK_ORDER[7] = 1
SUN_RANK_ORDER[8] = 2
SUN_RANK_ORDER[9] = 3
SUN_RANK_ORDER[10] = 7
SUN_RANK_ORDER[11] = 4
SUN_RANK_ORDER[12] = 5
SUN_RANK_ORDER[13] = 6

In [3]:
np.random.seed(42)

player_cards = np.array([rank + suit * 13 for suit in (0, 1, 2, 3) for rank in (1, 7, 8, 9, 10, 11, 12, 13)], dtype='uint8')
np.random.shuffle(player_cards)
player_cards = player_cards[:NUM_OF_STARTING_CARDS * 4].reshape(4, NUM_OF_STARTING_CARDS)


player_cards.sort()

player_cards

array([[14, 20, 26, 33, 40, 50],
       [ 1, 10, 23, 27, 46, 51],
       [ 7,  8, 11, 22, 24, 39],
       [ 9, 34, 37, 38, 47, 48]], dtype=uint8)

In [4]:
@njit
def get_suit(card):
    return (card - 1) // 13
    
@njit
def get_rank(card):
    return (card - 1) % 13 + 1

@njit
def has_suit(cards, suit):
    for card in cards:
        if get_suit(card) == suit:
            return True
    return False

@njit
def get_trick_winner(trick, starter, SUN_RANK_ORDER):
    trick_suit = get_suit(trick[0])
    winner = starter
    max_value = SUN_RANK_ORDER[get_rank(trick[0])]

    for i in range(1, 4):
        card = trick[i]
        if get_suit(card) == trick_suit:
            value = SUN_RANK_ORDER[get_rank(card)]
            if value > max_value:
                max_value = value
                winner = (starter + i) % 4

    return winner

@njit
def numba_get_legal_moves(current_cards, played_cards, num_played, current_player, num_starting_cards):
    moves = np.empty(num_starting_cards, dtype=np.uint8)
    count = 0
    hand = current_cards[current_player]

    if num_played % 4 == 0:  # starting a new trick
        for card in hand:
            if card != 0:
                moves[count] = card
                count += 1
    else:
        first_card = played_cards[(num_played // 4) * 4]
        trick_suit = get_suit(first_card)

        has_trick_suit = False
        for card in hand:
            if card != 0 and get_suit(card) == trick_suit:
                has_trick_suit = True
                break

        for card in hand:
            if card == 0:
                continue
            if (has_trick_suit and get_suit(card) == trick_suit) or not has_trick_suit:
                moves[count] = card
                count += 1

    return moves[:count]

@njit
def numba_calculate_score(played_cards, num_played, sun_scores, sun_rank_order):
    if num_played % 4 != 0:
        return -1

    total = 0
    cur_starter = 0

    for i in range(0, num_played, 4):
        trick = played_cards[i:i+4]
        rel_winner = get_trick_winner(trick, 0, sun_rank_order)
        cur_starter = (cur_starter + rel_winner) % 4

        round_score = 0
        for j in range(4):
            rank = get_rank(played_cards[i + j])
            round_score += sun_scores[rank]

        if cur_starter == 0 or cur_starter == 2:
            total += round_score

        if i == num_played - 4 and (cur_starter == 0 or cur_starter == 2):
            total += 10

    return total

In [5]:
class GameState:
    def __init__(self, current_cards):
        self.current_cards = current_cards
        self.played_cards = np.zeros(4 * NUM_OF_STARTING_CARDS, dtype='uint8')
        self.current_player = 0
        self.num_of_played_cards = 0
        self.card_indices = np.zeros(4 * NUM_OF_STARTING_CARDS, dtype='uint8') - 1
        self.player_indices = np.zeros(4 * NUM_OF_STARTING_CARDS, dtype='uint8') - 1

    def get_legal_moves(self):
        return numba_get_legal_moves(self.current_cards, self.played_cards, self.num_of_played_cards, self.current_player, NUM_OF_STARTING_CARDS)

    def choose(self, card):
        if self.num_of_played_cards == NUM_OF_STARTING_CARDS * 4:
            raise ValueError("Cannot choose a card since all cards have been played.")
            

        current_player_cards = self.current_cards[self.current_player]
        card_idx = np.where(current_player_cards == card)[0][0]
        
        self.played_cards[self.num_of_played_cards] = card
        current_player_cards[card_idx] = 0

        self.player_indices[self.num_of_played_cards] = self.current_player
        self.card_indices[self.num_of_played_cards] = card_idx

        # UPDATE TO NEXT PLAYER
        if self.num_of_played_cards % 4 == 3:
            current_trick_starter = self.player_indices[self.num_of_played_cards - (self.num_of_played_cards % 4)]
            self.current_player = get_trick_winner(self.played_cards[self.num_of_played_cards - 3: self.num_of_played_cards + 1], current_trick_starter, SUN_RANK_ORDER)
        else:
            self.current_player = (self.current_player + 1) % 4
        
        self.num_of_played_cards += 1

    def unchoose(self):
        if self.num_of_played_cards == 0:
            raise ValueError("Cannot unchoose a card since no cards have been played.")

        self.current_player = self.player_indices[self.num_of_played_cards - 1]
        self.player_indices[self.num_of_played_cards - 1] = 255

        card_idx = self.card_indices[self.num_of_played_cards - 1]
        self.card_indices[self.num_of_played_cards - 1] = 255
        
        self.current_cards[self.current_player][card_idx] = self.played_cards[self.num_of_played_cards - 1]
        self.played_cards[self.num_of_played_cards - 1] = 0
            
        self.num_of_played_cards -= 1

    def calculate_score(self):
        return numba_calculate_score(self.played_cards, self.num_of_played_cards, SUN_SCORES_ARRAY, SUN_RANK_ORDER)

In [6]:
def game_state_minimax(game_state, depth, alpha, beta, maximizing):
    global nodes_visited
    
    nodes_visited += 1
    
    if depth == 0 or game_state.num_of_played_cards == NUM_OF_STARTING_CARDS * 4:
        return game_state.calculate_score(), game_state.played_cards.copy()

    if maximizing:
        max_score = -float("inf")
        max_path = None
        for action in game_state.get_legal_moves():
            game_state.choose(action)
            score, path = game_state_minimax(game_state, depth - 1, alpha, beta, False if (game_state.current_player == 1 or game_state.current_player == 3) else True)
            if score > max_score:
                max_score = score
                max_path = path
            game_state.unchoose()
            alpha = max(alpha, score)
            if beta <= alpha:
                break
        return max_score, max_path
    
    else:
        min_score = float("inf")
        min_path = None
        for action in game_state.get_legal_moves():
            game_state.choose(action)
            score, path = game_state_minimax(game_state, depth - 1, alpha, beta, False if (game_state.current_player == 1 or game_state.current_player == 3) else True)
            if score < min_score:
                min_score = score
                min_path = path
            game_state.unchoose()
            beta = min(beta, score)
            if beta <= alpha:
                break
        return min_score, min_path

In [7]:
nodes_visited = 0

start = time.perf_counter()

game_state = GameState(player_cards.copy())

best_score, best_path = game_state_minimax(game_state, 4 * NUM_OF_STARTING_CARDS, -float("inf"), float("inf"), True)

end = time.perf_counter()

print(best_score)
print(best_path)
print(f"Time Elapsed: {end - start}")
print(f"Nodes Visited: {nodes_visited}")

72
[14 23 22  9 26  1 24 34 20 46  7 37 40 51 11 47 50 10 39 48 33 27  8 38]
Time Elapsed: 8.616057528997771
Nodes Visited: 730255


In [19]:
SUITS = {
    0: "Hearts",
    1: "Spades",
    2: "Diamonds",
    3: "Clubs",
}

print("Starting Cards:")
for i, card in enumerate(player_cards.reshape(NUM_OF_STARTING_CARDS * 4)):
    if i % NUM_OF_STARTING_CARDS == 0:
        print(f"\nPlayer #{i // NUM_OF_STARTING_CARDS + 1} Cards:")
    print(f"Card #{i + 1}: {get_rank(card)} of {SUITS[get_suit(card)]}")

print("\nBest Path:")
for i, card in enumerate(best_path):
    print(f"Card #{i + 1}: {get_rank(card)} of {SUITS[get_suit(card)]}")

Starting Cards:

Player #1 Cards:
Card #1: 1 of Spades
Card #2: 7 of Spades
Card #3: 13 of Spades
Card #4: 7 of Diamonds
Card #5: 1 of Clubs
Card #6: 11 of Clubs

Player #2 Cards:
Card #7: 1 of Hearts
Card #8: 10 of Hearts
Card #9: 10 of Spades
Card #10: 1 of Diamonds
Card #11: 7 of Clubs
Card #12: 12 of Clubs

Player #3 Cards:
Card #13: 7 of Hearts
Card #14: 8 of Hearts
Card #15: 11 of Hearts
Card #16: 9 of Spades
Card #17: 11 of Spades
Card #18: 13 of Diamonds

Player #4 Cards:
Card #19: 9 of Hearts
Card #20: 8 of Diamonds
Card #21: 11 of Diamonds
Card #22: 12 of Diamonds
Card #23: 8 of Clubs
Card #24: 9 of Clubs

Best Path:
Card #1: 1 of Spades
Card #2: 10 of Spades
Card #3: 9 of Spades
Card #4: 9 of Hearts
Card #5: 13 of Spades
Card #6: 1 of Hearts
Card #7: 11 of Spades
Card #8: 8 of Diamonds
Card #9: 7 of Spades
Card #10: 7 of Clubs
Card #11: 7 of Hearts
Card #12: 11 of Diamonds
Card #13: 1 of Clubs
Card #14: 12 of Clubs
Card #15: 11 of Hearts
Card #16: 8 of Clubs
Card #17: 11 of 